<a href="https://colab.research.google.com/github/akimi-yano/data-science/blob/main/LLM_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### <font color=blue>Installing packages</font>



In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain pypdf rapidocr-onnxruntime streamlit pdf2image pdfminer.six pikepdf langchain_experimental


In [ ]:
!pip install pi_heif unstructured_inference

In [ ]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!apt-get install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


#### <font color=blue>OpenAI API


In [ ]:
#OpenAI API key
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_KEY')

In [ ]:
#Setup LangSmith to trace development
from langsmith import Client
os.environ["LANGCHAIN_PROJECT"] = 'RAG_LAB'
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH')
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## <font color=blue>Creating ChatBot</font>


#### Define models

In [ ]:
GPT4 = 'gpt-4-0125-preview'
GPT3 = 'gpt-3.5-turbo-0125'

#### Simple ChatBot, no memory

In [ ]:
#Import ChatOpenAI class
from langchain_openai import ChatOpenAI


In [ ]:
#Define the LLM. Specify model
Chat = ChatOpenAI(model = GPT4)

In [ ]:
# Invoke the chat with simple question to test it out
Chat.invoke('What is your knowledge cut off day?')

AIMessage(content="My knowledge is up to date as of September 2021. I can't provide information on events or developments that have occurred after that time.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 15, 'total_tokens': 44, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4-0125-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f1c14e39-1a9d-4fa2-8048-52727345fc4c-0', usage_metadata={'input_tokens': 15, 'output_tokens': 29, 'total_tokens': 44})

#### Adding memory and memory management

In [ ]:
# Import ChatMessageHistory class that will store chat history
# Import chat prompt templates classes and Message placeholders classes
from langchain.memory import ChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
# Initialize a new ChatMessageHistory object
chat_history = ChatMessageHistory()

In [ ]:
# Add a user message to the chat history
chat_history.add_user_message("What day ChatGPT was launched")

In [ ]:
# Add an AI response message to the chat history
chat_history.add_ai_message("ChatGPT was launched at November 30, 2022")


In [ ]:
# Access the messages property of the chat_history object
chat_history.messages

[HumanMessage(content='What day ChatGPT was launched', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ChatGPT was launched at November 30, 2022', additional_kwargs={}, response_metadata={})]

In [ ]:
# Add another user message to the chat history
chat_history.add_user_message("Was it a successful launch?")

In [ ]:
# Create a ChatPromptTemplate using messages
prompt = ChatPromptTemplate.from_messages(
    [
        # Define a system message as a tuple
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        # Add a placeholder for the chat messages
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
# Create a simple Chain by passing prompt to LLM
        # input -> container - input goes into here
Chain = prompt | Chat

In [ ]:
#Invoking simple chain from messages
Chain.invoke({"messages": chat_history.messages})

AIMessage(content='Yes, the launch of ChatGPT was highly successful. It quickly captured widespread attention due to its advanced natural language processing capabilities, allowing users to engage in human-like conversations, generate text based on prompts, and even create content across various formats and genres. The platform received praise for its ability to understand and generate human-like text, making it a useful tool for a wide range of applications. This success was reflected in its rapid user adoption, with millions of users trying it out within a few weeks of its launch. The buzz around ChatGPT also sparked discussions about the potential and implications of advanced AI in various sectors, including education, customer service, and content creation.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 61, 'total_tokens': 195, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4-0125-previ

#### Creating a loop to run Chat with history, printable user imputs and chat outputs

In [ ]:
#Import
from langchain_core.runnables.history import RunnableWithMessageHistory
#Define stop words for our chatbot
stop_words = ["exit", "quit", "stop"]

In [ ]:
#Define chat history
chat_history = ChatMessageHistory()

#Define LLM

Chat = ChatOpenAI(model = GPT4)

# Create a ChatPromptTemplate using messages
prompt = ChatPromptTemplate.from_messages(
    [
        # Define a system message as a tuple
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        # Add a placeholder for the chat messages
        MessagesPlaceholder(variable_name="messages"),
    ]
)

#Define the chain
Chat_chain = prompt | Chat

#Use RunnableWithMessageHistory as a wrapper to manage message history
Chain_with_message_history = RunnableWithMessageHistory(
    Chat_chain,
    #define access to chat history
    lambda session_id : chat_history,
    input_messages_key="messages",
    history_messages_key="chat_history"
)

# Perform chat turns
print("Starting the chat...")
while True:
    question = input("User: ")

    # Check if the user input matches a stop word
    if question.lower() in stop_words:
        print("Exiting the chat...")
        break

    # Add a user message to the chat history
    chat_history.add_user_message(question)

    #Generate AI response
    ai_response = Chain_with_message_history.invoke({"messages": chat_history.messages}, {"configurable": {"session_id": chat_history}})

    # Add an AI response message to the chat history
    chat_history.add_ai_message(ai_response.content)

    #Display AI answer
    print(f"AI: {ai_response.content}")

Starting the chat...
User: exit
Exiting the chat...


## <font color=blue>Building RAG Chatbot</font>

#### <font color=blue>Load documents</font>

##### Grab/download Cognition is all you need paper from [arxiv.org](https://arxiv.org/abs/2403.02164)

In [ ]:
#Import pdf loader.
# from langchain_community.document_loaders import UnstructuredPDFLoader

In [ ]:
#Define loader
# loader_pdf = UnstructuredPDFLoader("/content/Cognition is All You Need - Article.pdf", extract_images=True)

In [ ]:
#Load an article
# article_pdf = loader_pdf.load()

In [ ]:
#Print doc to check it out
# print(article_pdf)

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Cognition is All You Need - Article.pdf")
pages = loader.load_and_split()

In [ ]:
print(pages)

[Document(metadata={'source': '/content/Cognition is All You Need - Article.pdf', 'page': 0}, page_content='C o g n i t i o nisAllYouNeedTheNextLayerofAIAboveLargeLanguageModels\nPre-Public ationPositionPaperDraft1.1March4,2024,ForCommen ts\nNovaSpivack1,SamDouglas1,MichelleCrames1,TimConnor s1\n1Mindc orp,Inc(www.mindc orp.ai)contact@mindc orp.aiwww.mindc orp.aiwww.linkedin.c om/compan y/mindc orp-aitwitter.com/mindc orpai'), Document(metadata={'source': '/content/Cognition is All You Need - Article.pdf', 'page': 1}, page_content='Contents\nAbstract...................................................................................................................................2\nIntroduction\n..................................................................................................................................\n2RelatedResearch.....................................................................................................................5DefiningConversationalAI......

#### <font color=blue>Split the document</font>

In [ ]:
# Import text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create an instance of RecursiveCharacterTextSplitter with custom chunk size and overlap
chunk_size = 750  # Adjust the chunk size as needed
chunk_overlap = 0  # Set the overlap between chunks

#Initiate splitter with desired parameters
splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

In [ ]:
# Split the document into chunks using the RecursiveCharacterTextSplitter
# splits = splitter.split_documents(article_pdf)
splits = splitter.split_documents(pages)

In [ ]:
# Print the number of splits in the doc
print(len(splits))

258


In [ ]:
# Print each split and a separator for readability
for split in splits:
    print(split)
    print("---")

page_content='C o g n i t i o nisAllYouNeedTheNextLayerofAIAboveLargeLanguageModels
Pre-Public ationPositionPaperDraft1.1March4,2024,ForCommen ts
NovaSpivack1,SamDouglas1,MichelleCrames1,TimConnor s1
1Mindc orp,Inc(www.mindc orp.ai)contact@mindc orp.aiwww.mindc orp.aiwww.linkedin.c om/compan y/mindc orp-aitwitter.com/mindc orpai' metadata={'source': '/content/Cognition is All You Need - Article.pdf', 'page': 0}
---
page_content='Contents
Abstract...................................................................................................................................2
Introduction
..................................................................................................................................' metadata={'source': '/content/Cognition is All You Need - Article.pdf', 'page': 1}
---
page_content='2RelatedResearch.....................................................................................................................5DefiningConversationalAI.............

#### <font color=blue>Create embeddings</font>

In [ ]:
#Import vectorstore database and embeddings model
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Embeddings model
embeddings_model = OpenAIEmbeddings

In [ ]:
#Define vector DB. Run this line of code only once.
#If accidently did more delete DB

vector_db = Chroma.from_documents(documents=splits, embedding=embeddings_model())

In [ ]:
#Code to delete db. (if needed)

# Delete the collection
# vector_db.delete_collection()
# print("Collection deleted successfully.")

#### <font color=blue>Define Retriever</font>

In [ ]:
retriever = vector_db.as_retriever()

#### <font color=blue>Test retriever</font>

In [ ]:
#Define question
question = 'What are functional requirements for cognitive AI'

#Fetch 3 documents from vector store related to question
vector_db.similarity_search_with_score(question, k=3)

[(Document(metadata={'page': 15, 'source': '/content/Cognition is All You Need - Article.pdf'}, page_content="Figure6.ConversationalVersusCognitiv eAIQuadrants.\nCognitiveAIFunctionalArchitecture\nCognitiv eAIrepresen tsaparadigmshift,movingbeyondtheconfinesofConversationalAI'srelianceonprobabilis ticreasoningsimula tionstoactualprogramma ticreasoning.Thisshiftisembodiedinadual-la yerarchitectur ethatelevatesreasoning ,self-impr ovemen t,andadap tabilitytosecond-or derintelligence,fundamen tallydistinguishingCognitiv eAIfromitspredecessor s.Belo wwewilldiscussthefunctionalarchitectur eandformalrequir emen tsforCognitiv eAIsystems.\nFunctionalRequirementsforCognitiveAI\nToqualif yasCognitiv eAI,asystemmus tbearchitectedtomee tthefollowingfunctionalcriteria:"),
  0.2199014574289322),
 (Document(metadata={'page': 1, 'source': '/content/Cognition is All You Need - Article.pdf'}, page_content='eAIFunctionalArchitecture.........................................................................

#### <font color=blue>Biuld chain that will answer queestions over defined docs</font>

In [ ]:
from langchain.prompts import ChatPromptTemplate
# Prompt
template = """Answer the question based on the following context:
{context}

Question: {question}
"""

#Define rag_prompt from template
rag_prompt = ChatPromptTemplate.from_template(template)

#Print the promt to check it everything is ok
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [ ]:
#Define LLM

In [ ]:
RAG_llm = ChatOpenAI(model=GPT4)

In [ ]:
#Define Chain

In [ ]:
RAG_chain = rag_prompt | RAG_llm

In [ ]:
#Assign docs
docs = vector_db.similarity_search(question, k=3)

In [ ]:
#Chain to answer question based on defined docs
RAG_chain.invoke({"context":docs,"question": question})

AIMessage(content='The document provides a context for understanding the functional architecture and requirements for Cognitive AI but does not enumerate the specific functional requirements directly in the provided text excerpts. The mention of "Functional Requirements for Cognitive AI" suggests that there is a detailed discussion or list of these requirements in the document, specifically on page 15, where the architecture and requirements are discussed. \n\nHowever, based on the context given and the nature of Cognitive AI as described, we can infer some potential functional requirements:\n\n1. **Programmatic Reasoning Capability**: The ability to go beyond probabilistic reasoning simulations to actual programmatic reasoning, implying a system that can engage with logic and structured decision-making processes in a manner that mimics human cognitive abilities.\n\n2. **Dual-layer Architecture**: This suggests that Cognitive AI systems must have a foundational structure that supports 

#### <font color=blue>Composing the Retrieval-Augmented Generation Chain with dynamic retrieval</font>

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# Create the Retrieval-Augmented Generation (RAG) chain with dynamic retrieval
rag_chain = (
    # Define the input variables for the chain
    {"context": retriever, "question": RunnablePassthrough()}
    # Pipe the input through the RAG prompt template
    | rag_prompt
    # Pass the formatted prompt to the language model (LLM)
    | RAG_llm
    # Parse the LLM's output using the StrOutputParser
    | StrOutputParser()
)

In [ ]:
#Invoke the chain

rag_chain.invoke("What lies at the core of Cognitive AI's functional architecture?")


"At the core of Cognitive AI's functional architecture is an intelligence stack comprising two critical layers: a Cognitive Layer and a Conversational Layer."

#### <font color=blue>Build RAG BOT chain prompt template</font>

In [ ]:
#Import required classes for prompt template

from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage

# Define the human prompt template
# Here we making sure that context will be passed to LLM before question will be added by human.
human_prompt = """Answer the question based on the following context: {context}

"""

In [ ]:
# Initialize the chat history
chat_history = ChatMessageHistory()

In [ ]:
# Define the question
question = "What lies at the core of Cognitive AI's functional architecture?"

In [ ]:
# Retrieve relevant context based on the question
context = vector_db.similarity_search(question, k=3)

In [ ]:
# Create a PromptTemplate from the human prompt
prompt_template = PromptTemplate.from_template(human_prompt)

In [ ]:
# Format the prompt with the retrieved context and question
formatted_prompt = prompt_template.format(context=context)

In [ ]:
# Create a HumanMessage with the formatted prompt
formatted_human_message = [HumanMessage(content=formatted_prompt)]

In [ ]:
# Define the RAG prompt template
rag_bot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions over the documents to the best of your ability."),
        *formatted_human_message,
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [ ]:
# Create the RAG LLM chain by piping the RAG prompt to the LLM
rag_bot_chain = rag_bot_prompt | RAG_llm

In [ ]:
#Define stop words for our chatbot
stop_words = ["exit", "quit", "stop"]

chat_history = ChatMessageHistory()


rag_chain_with_message_history = RunnableWithMessageHistory(
    rag_bot_chain,
    lambda session_id : chat_history,
    input_messages_key="messages",
    history_messages_key="chat_history"
)

# Perform chat turns
print("Starting the chat...")
while True:
    question = input("User: ")

    # Check if the user input matches a stop word
    if question.lower() in stop_words:
        print("Exiting the chat...")
        break

    # Retrieve relevant context based on the question
    context = vector_db.similarity_search(question, k=3)

    # Add a user message to the chat history
    chat_history.add_user_message(question)

    #Generate AI response
    ai_response = rag_chain_with_message_history.invoke({"messages": chat_history.messages}, {"configurable": {"session_id": chat_history}})

    # Add an AI response message to the chat history
    chat_history.add_ai_message(ai_response.content)

    #Display AI answer
    print(f"AI: {ai_response.content}")

Starting the chat...
User: Explain to me as I'm 5 what is Neuro-Symbolic Reasoning?
AI: Alright, imagine your brain is like a super smart detective that's really good at solving puzzles. Now, let's say you have two types of puzzles. One kind is like a big, colorful picture puzzle (we'll call these "symbols" because each piece is distinct and means something specific). The other type is like a guessing game where you try to figure out a pattern (this is the "neuro" part, which works a lot like guessing or learning patterns).

Neuro-Symbolic Reasoning is when your detective brain decides to use both types of puzzles to solve a mystery. It combines the colorful picture pieces (symbols) with the pattern-guessing game (neuro) to make solving the mystery easier and faster. This way, the detective doesn't just guess; it also uses specific clues (symbols) to be really sure about the answer.
User: stop
Exiting the chat...


## <font color=blue>RAG Chatbot over webpage</font>

#### <font color=blue>Load the website</font>

In [ ]:
# Import the WebBaseLoader class from the langchain_community.document_loaders module
from langchain_community.document_loaders import WebBaseLoader

# Create an instance of the WebBaseLoader class with the specified URL
loader = WebBaseLoader("https://huggingface.co/")


In [ ]:
# Load the webpage data using the WebBaseLoader instance
webpage_data = loader.load()

In [ ]:
#Check results
webpage_data

[Document(metadata={'source': 'https://huggingface.co/', 'title': 'Hugging Face ‚Äì The AI community building the future.', 'description': 'We‚Äôre on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \nHugging Face ‚Äì The AI community building the future.\n\n\n\n\n\n\n\n\nHugging Face\n\n\n\n\n\n\n\n\t\t\t\t\tModels\n\n\t\t\t\t\tDatasets\n\n\t\t\t\t\tSpaces\n\n\t\t\t\t\tPosts\n\n\t\t\t\t\tDocs\n\n\n\n\n\t\t\tSolutions\n\t\t\n\nPricing\n\t\t\t\n\n\n\n\n\n\nLog In\n\t\t\t\t\nSign Up\n\t\t\t\t\t\n\nNEW\n\t\t\t\t\nAI Tools are now available in HuggingChat\n\t\t\t\t\n\nThe AI community building the future.\n\t\t\t\nThe platform where the machine learning community collaborates on models, datasets, and applications.\n\t\t\t\n\n\n\n\n\n\n\t\t\tTrending onthis week\n\t\t\t\n\n\n\t\t\t\t\tModels\n\t\t\t\t\n\nstepfun-ai/GOT-OCR2_0\n\n\nUpdated\n\t\t\t

In [ ]:
# Set the chunk size and overlap for splitting the webpage data
web_chunk_size = 400  # Adjust the chunk size as needed
web_chunk_overlap = 0

# Create a RecursiveCharacterTextSplitter instance with the specified chunk size and overlap
web_splitter = RecursiveCharacterTextSplitter(chunk_size=web_chunk_size, chunk_overlap=web_chunk_overlap)

In [ ]:
# Split the Markdown document into chunks based on paragraphs
webpage_splits = web_splitter.split_documents(webpage_data)


In [ ]:
#Check number of splits
print(len(webpage_splits))


13


In [ ]:
# Print the resulting splits
for split in webpage_splits:
    print(split)
    print("---")

page_content='Hugging Face ‚Äì The AI community building the future.








Hugging Face







					Models

					Datasets

					Spaces

					Posts

					Docs




			Solutions
		

Pricing
			






Log In
				
Sign Up
					

NEW
				
AI Tools are now available in HuggingChat' metadata={'source': 'https://huggingface.co/', 'title': 'Hugging Face ‚Äì The AI community building the future.', 'description': 'We‚Äôre on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.'}
---
page_content='The AI community building the future.
			
The platform where the machine learning community collaborates on models, datasets, and applications.
			






			Trending onthis week
			


					Models
				

stepfun-ai/GOT-OCR2_0


Updated
					9 days ago
‚Ä¢ 

					134k
				
				‚Ä¢ 

					778

				

black-forest-labs/FLUX.1-dev


Updated
					Aug 16
‚Ä¢ 

					924k
				
				‚Ä¢ 

					4.8k' metadata={'source': 'https://huggingfa

#### <font color=blue>Vectorstore</font>

In [ ]:
#Delete previous vectorstore collection
# Delete the collection
vector_db.delete_collection()
print("Collection deleted successfully.")

Collection deleted successfully.


In [ ]:
# Create an instance of the OpenAIEmbeddings model
web_embeddings_model = OpenAIEmbeddings()

# Create a Chroma vector database from the webpage splits using the OpenAIEmbeddings model
vector_db_webpage = Chroma.from_documents(documents=webpage_splits, embedding=web_embeddings_model)

#### <font color=blue>Define and test retrieval</font>

In [ ]:
# Create a retriever from the Chroma vector database
web_retriever = vector_db_webpage.as_retriever()

In [ ]:
# Define a question to search for in the vector database
question = 'Explain me about HuggingFace and its technical capabilities?'

# Perform a similarity search in the vector database using the question
search_results = vector_db_webpage.similarity_search(question, k=4)

In [ ]:
print(search_results)

[Document(metadata={'description': 'We‚Äôre on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.', 'source': 'https://huggingface.co/', 'title': 'Hugging Face ‚Äì The AI community building the future.'}, page_content='Hugging Face ‚Äì The AI community building the future.\n\n\n\n\n\n\n\n\nHugging Face\n\n\n\n\n\n\n\n\t\t\t\t\tModels\n\n\t\t\t\t\tDatasets\n\n\t\t\t\t\tSpaces\n\n\t\t\t\t\tPosts\n\n\t\t\t\t\tDocs\n\n\n\n\n\t\t\tSolutions\n\t\t\n\nPricing\n\t\t\t\n\n\n\n\n\n\nLog In\n\t\t\t\t\nSign Up\n\t\t\t\t\t\n\nNEW\n\t\t\t\t\nAI Tools are now available in HuggingChat'), Document(metadata={'description': 'We‚Äôre on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.', 'source': 'https://huggingface.co/', 'title': 'Hugging Face ‚Äì The AI community building the future.'}, page_content='Enterprise\nGive your team the most adv

#### <font color=blue>Build a RAG bot over webpage</font>

In [ ]:
# Define a list of stop words
stop_words = ["exit", "quit", "stop"]

# Initialize the chat history
chat_history = ChatMessageHistory()

# Create a RunnableWithMessageHistory instance
rag_chain_with_message_history = RunnableWithMessageHistory(
    rag_bot_chain,
    lambda session_id: chat_history,
    input_messages_key="messages",
    history_messages_key="chat_history",
)

# Perform chat turns
print("Starting the chat...")
while True:
    # Get user input
    question = input("User: ")

    # Retrieve relevant context based on the question
    context = vector_db_webpage.similarity_search(question, k=4)
    print(context)

    # Check if the user input matches a stop word
    if question.lower() in stop_words:
        print("Exiting the chat...")
        break

    # Add the user question to the history
    chat_history.add_user_message(question)

    # Get the AI response using the RAG chain with message history
    ai_response = rag_chain_with_message_history.invoke(
        {"messages": chat_history.messages},
        {"configurable": {"session_id": chat_history}}
    )

    # Add the AI response to the history
    chat_history.add_ai_message(ai_response.content)

    # Display the AI's answer
    print(f"AI: {ai_response.content}")

Starting the chat...
User: Explain to me like I'm 8 years old. What is Hugging Face?
[Document(metadata={'description': 'We‚Äôre on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.', 'source': 'https://huggingface.co/', 'title': 'Hugging Face ‚Äì The AI community building the future.'}, page_content='Hugging Face ‚Äì The AI community building the future.\n\n\n\n\n\n\n\n\nHugging Face\n\n\n\n\n\n\n\n\t\t\t\t\tModels\n\n\t\t\t\t\tDatasets\n\n\t\t\t\t\tSpaces\n\n\t\t\t\t\tPosts\n\n\t\t\t\t\tDocs\n\n\n\n\n\t\t\tSolutions\n\t\t\n\nPricing\n\t\t\t\n\n\n\n\n\n\nLog In\n\t\t\t\t\nSign Up\n\t\t\t\t\t\n\nNEW\n\t\t\t\t\nAI Tools are now available in HuggingChat'), Document(metadata={'description': 'We‚Äôre on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.', 'source': 'https://huggingface.co/', 'title': 'Hugging Face ‚Äì The AI co